<a href="https://colab.research.google.com/github/datascience-uniandes/data-quality-tutorial/blob/master/data-quality-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Quality tutorial

MINE-4101: Applied Data Science  
Univerisdad de los Andes  
  
Dataset: Homicides Colombia  
Source: [datos.gov.co](datos.gov.co)
  
Last update: September, 2022

In [1]:
!pip install pylev

In [2]:
# Importing required libraries

import re
from random import randint
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import pylev

In [3]:
# Parameter for showing all columns when printing a dataframe
pd.set_option('display.max_columns', None)

In [4]:
# Loading data
homicides_df = pd.read_csv("./data/homicides.csv")

In [5]:
# Printing the dataset dimensions
homicides_df.shape

(12400, 22)

In [6]:
# Printing column data types
homicides_df.dtypes

FECHA                 object
DEPARTAMENTO          object
MUNICIPIO             object
DIA                   object
HORA                  object
BARRIO                object
ZONA                  object
CLASE DE SITIO        object
ARMA O MEDIO          object
MOVIL VICTIMA         object
MOVIL AGRESOR         object
EDAD                 float64
GENERO                object
ESTADO CIVIL          object
CLASE EMPLEADO        object
PROFESION             object
ESCOLARIDAD           object
PAIS NACE             object
CODIGO DANE          float64
AÑO DE NACIMIENTO    float64
CÉDULA                object
CORREO                object
dtype: object

In [7]:
homicides_df.head()

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
0,01/01/2021 12:00:00 AM,ANTIOQUIA,AMAGÁ,Jueves,6:00,EL VOLCAN,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,44.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5030000.0,1971.0,42-908,agbnqg2122@unidatos.edu.co
1,01/01/2021 12:00:00 AM,ANTIOQUIA,BARBOSA,Jueves,9:00,VDA. MATASANOS,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,NO REPORTADO,5079000.0,1985.0,15-183,rbkeui3584@gmail.com
2,01/01/2021 12:00:00 AM,ANTIOQUIA,EL BAGRE,Jueves,19:00,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1982.0,84-786,aorkhf9155@unidatos.edu.co
3,01/01/2021 12:00:00 AM,ANTIOQUIA,JARDÍN,Jueves,11:20,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1975.0,31-289,dhtemr6623@unidatos.edu.co
4,01/01/2021 12:00:00 AM,ANTIOQUIA,MEDELLÍN (CT),Juees,15:00,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1949.0,66-363,artatj9268@unidatos.edu.co


In [8]:
# Creating a lambda expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")

In [9]:
# Creating a function for validating which value is causing the previous error
def error_in_format(x):
    try:
        datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")
        return False
    except:
        return True

In [10]:
# Using the function for validation
homicides_df.loc[homicides_df["FECHA"].apply(error_in_format)]

# THE LAST ROW IS CAUSING THE ERROR

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
12399,TOTAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Deleting a row by its index
homicides_df.drop(12399, inplace = True)

In [12]:
# Trying to parse the datetime string again
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [13]:
# Counting homicides by hour
homicides_df["FECHA"].dt.hour.value_counts()

# All datetime hour parts are the same

12    12399
Name: FECHA, dtype: int64

In [14]:
# The homicide hour is available in a different column!
# Merging both columns
homicides_df["FECHA"] = homicides_df["FECHA"].astype(str).apply(lambda x: x[:11]) + homicides_df["HORA"]

In [15]:
del homicides_df["HORA"]

In [16]:
homicides_df["FECHA"].head()

0     2021-01-01 6:00
1     2021-01-01 9:00
2    2021-01-01 19:00
3    2021-01-01 11:20
4    2021-01-01 15:00
Name: FECHA, dtype: object

In [17]:
# Making a new expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M")

In [18]:
# Applying the expression
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [19]:
homicides_df.dtypes

FECHA                datetime64[ns]
DEPARTAMENTO                 object
MUNICIPIO                    object
DIA                          object
BARRIO                       object
ZONA                         object
CLASE DE SITIO               object
ARMA O MEDIO                 object
MOVIL VICTIMA                object
MOVIL AGRESOR                object
EDAD                        float64
GENERO                       object
ESTADO CIVIL                 object
CLASE EMPLEADO               object
PROFESION                    object
ESCOLARIDAD                  object
PAIS NACE                    object
CODIGO DANE                 float64
AÑO DE NACIMIENTO           float64
CÉDULA                       object
CORREO                       object
dtype: object

In [20]:
homicides_df.head()

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
0,2021-01-01 06:00:00,ANTIOQUIA,AMAGÁ,Jueves,EL VOLCAN,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,44.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5030000.0,1971.0,42-908,agbnqg2122@unidatos.edu.co
1,2021-01-01 09:00:00,ANTIOQUIA,BARBOSA,Jueves,VDA. MATASANOS,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,NO REPORTADO,5079000.0,1985.0,15-183,rbkeui3584@gmail.com
2,2021-01-01 19:00:00,ANTIOQUIA,EL BAGRE,Jueves,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1982.0,84-786,aorkhf9155@unidatos.edu.co
3,2021-01-01 11:20:00,ANTIOQUIA,JARDÍN,Jueves,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1975.0,31-289,dhtemr6623@unidatos.edu.co
4,2021-01-01 15:00:00,ANTIOQUIA,MEDELLÍN (CT),Juees,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1949.0,66-363,artatj9268@unidatos.edu.co


In [21]:
# Creating a dictionary representing the valid departments for Colombia
departments_list = ['ANTIOQUIA', 'ATLÁNTICO', 'BOLÍVAR', 'BOYACÁ', 'CALDAS', 'CAQUETÁ',
       'CASANARE', 'CAUCA', 'CESAR', 'CHOCÓ', 'CÓRDOBA', 'META',
       'CUNDINAMARCA', 'HUILA', 'MAGDALENA', 'NARIÑO', 'PUTUMAYO',
       'RISARALDA', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE',
       'NORTE DE SANTANDER', 'GUAJIRA', 'QUINDÍO', 'SAN ANDRÉS Y PROVIDENCIA', 'ARAUCA',
       'GUAINÍA', 'VICHADA', 'VAUPÉS', 'GUAVIARE', 'AMAZONAS']

In [22]:
# Finding values not matching with the dictionary
homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

array(['SAN ANDRÉS', 'N. DE SANTANDER'], dtype=object)

<span style="color:red">TODO: Replace the values identified as error to a valid value from the dictionary</span>

<span style="color:red">Hint: You can use the replace() pandas function</span>

In [23]:
# Copio el DF para no alterar el original

homicides_df['DEPARTAMENTO'].replace('SAN ANDRÉS Y PROVIDENCIA','SAN ANDRÉS', inplace=True)
homicides_df['DEPARTAMENTO'].replace('SAN ANDRÉS','SAN ANDRÉS Y PROVIDENCIA', inplace=True)
homicides_df['DEPARTAMENTO'].replace('N. DE SANTANDER','NORTE DE SANTANDER', inplace=True)

# Verifico que no esten datos por fuera de la serie 

homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

array([], dtype=object)

In [24]:
# Detecting duplicates by "CÉDULA" column
duplicates_by_cedula = homicides_df.loc[homicides_df["CÉDULA"].duplicated(keep = False)]

In [25]:
duplicates_by_cedula.shape

(1627, 21)

In [26]:
# Showing some examples
duplicates_by_cedula.sort_values("CÉDULA", ascending = True).head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
4041,2021-05-03 23:30:00,CAUCA,EL TAMBO,Domingo,LA VICTORIA,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,19256000.0,1970.0,10-048,aitufn1227@gmail.com
3810,2021-04-26 03:00:00,ANTIOQUIA,SAN JERÓNIMO,Domingo,LA PLAYA,URBANA,"HOTELES, RESIDENCIAS, Y SIMILARES.",CUERDA/SOGA/CADENA,A PIE,A PIE,25.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5656000.0,1990.0,10-048,ibbcpu2509@unidatos.edu.co
693,2021-01-19 05:30:00,ANTIOQUIA,SALGAR,Lunes,LA HABANA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,58.0,MASCULINO,SOLTERO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5642000.0,57.0,10-079,unhoqj1172@unidatos.edu.co
10796,2021-11-20 03:00:00,VALLE,CALI (CT),Viernes,POTRERO GRANDE E21,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,21.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1994.0,10-079,iknnoj8430@unidatos.edu.co
2386,2021-03-14 02:00:00,META,FUENTE DE ORO,Sábado,VEREDA PUERTO NUEVO,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA BLANCA,A PIE,A PIE,42.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,50287000.0,1973.0,10-255,afompq7113@unidatos.edu.co
138,2021-01-02 06:20:00,CAUCA,SOTARA,Viernes,CENTRO,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,22.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,19760000.0,1993.0,10-255,cobgqs8819@unidatos.edu.co


<span style="color:red">TODO: Delete records with "CÉDULA" duplicated</span>

<span style="color:red">Hint: You can use drop_duplicates() pandas function.</span>

In [27]:
# No estoy de acuerdo con borrar porque son personas diferentes en la mayoria de casos, a pesar que tienen la misma cédula son registros diferentes. 

# Yo haría lo siguiente:
copyHomicides_df = duplicates_by_cedula
copyHomicides_df.drop_duplicates(keep='first', inplace=True)
copyHomicides_df.shape

# Como se evidencia, no hay borrado de datos. Ahora, lo solicitado: 
duplicates_by_cedula.drop_duplicates(subset='CÉDULA',keep='first', inplace=True)
duplicates_by_cedula.shape

#Con esto quedan en 791 registros únicamente.

(791, 21)

In [28]:
# Using regular expressions for validating if "CÉDULA" values match the pattern XX-XXX
cedula_malformed = homicides_df.loc[homicides_df["CÉDULA"].apply(lambda x: (re.match("\d{2}-\d{3}", x) is None))]

In [29]:
cedula_malformed.shape

(28, 21)

In [30]:
cedula_malformed.head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
172,2021-01-03 11:00:00,META,MESETAS,Sábado,VEREDA EL CAFRE,RURAL,ZONA SELVÁTICA,MINA ANTIPERSONA,A PIE,A PIE,24.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,50330000.0,1991.0,680-21,oohghd8899@gmail.com
1114,2021-02-01 12:30:00,ATLÁNTICO,BARRANQUILLA (CT),Domingo,LA LUZ,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,29.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,8001000.0,1986.0,140-17,correo5853@colombia.gov.co
2119,2021-03-06 15:30:00,VALLE,CALI (CT),Viernes,QUINTAS DEL SOL E14,URBANA,DENTRO DE LA VIVIENDA,ARMA BLANCA,A PIE,A PIE,20.0,FEMENINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1995.0,975-31,ohbqrk3631@unidatos.edu.co
3309,2021-04-11 10:30:00,CAQUETÁ,FLORENCIA (CT),Sávado,VIA MORELIA,RURAL,CARCELES,CORTANTES,A PIE,A PIE,23.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,18001000.0,1992.0,348-66,pdkqur8407@unidatos.edu.co
3409,2021-04-13 23:20:00,CÓRDOBA,SAHAGÚN,Lunes,CORREGIMIENTO DE BAJO GRANDE,URBANA,BILLARES,CONTUNDENTES,A PIE,A PIE,49.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,23660000.0,1966.0,496-18,fdbbeo6751@unidatos.edu.co
3588,2021-04-19 20:00:00,BOLÍVAR,CARTAGENA (CT),Domingo,LA ESPERANZA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,NO REPORTADO,NO REPORTADO,13001000.0,1970.0,188-03,diebuo5651@unidatos.edu.co


<span style="color:red">TODO: Fix the malformed "CÉDULA" values.</span>

In [31]:
# Parto de la base que para corregir la Cédula se debe correr el guión un caracter. Por esto copio el dataSet para hacer la transformación quitando el guión y 
# e insertando el guión en la posición 3.

dfPrueba = cedula_malformed
dfPrueba['CÉDULA'].replace('-','',regex=True,inplace=True)
dfPrueba['CÉDULA1']=dfPrueba['CÉDULA'].str[:2]
dfPrueba['CÉDULA2']=dfPrueba['CÉDULA'].str[-3:]
cedula_malformed['CÉDULA']=dfPrueba['CÉDULA1']+'-'+dfPrueba['CÉDULA2']

In [32]:
cedula_malformed

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO,CÉDULA1,CÉDULA2
172,2021-01-03 11:00:00,META,MESETAS,Sábado,VEREDA EL CAFRE,RURAL,ZONA SELVÁTICA,MINA ANTIPERSONA,A PIE,A PIE,24.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,50330000.0,1991.0,68-021,oohghd8899@gmail.com,68,021
1114,2021-02-01 12:30:00,ATLÁNTICO,BARRANQUILLA (CT),Domingo,LA LUZ,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,29.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,8001000.0,1986.0,14-017,correo5853@colombia.gov.co,14,017
2119,2021-03-06 15:30:00,VALLE,CALI (CT),Viernes,QUINTAS DEL SOL E14,URBANA,DENTRO DE LA VIVIENDA,ARMA BLANCA,A PIE,A PIE,20.0,FEMENINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1995.0,97-531,ohbqrk3631@unidatos.edu.co,97,531
3309,2021-04-11 10:30:00,CAQUETÁ,FLORENCIA (CT),Sávado,VIA MORELIA,RURAL,CARCELES,CORTANTES,A PIE,A PIE,23.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,18001000.0,1992.0,34-866,pdkqur8407@unidatos.edu.co,34,866
3409,2021-04-13 23:20:00,CÓRDOBA,SAHAGÚN,Lunes,CORREGIMIENTO DE BAJO GRANDE,URBANA,BILLARES,CONTUNDENTES,A PIE,A PIE,49.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,23660000.0,1966.0,49-618,fdbbeo6751@unidatos.edu.co,49,618
3588,2021-04-19 20:00:00,BOLÍVAR,CARTAGENA (CT),Domingo,LA ESPERANZA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,NO REPORTADO,NO REPORTADO,13001000.0,1970.0,18-803,diebuo5651@unidatos.edu.co,18,803
4565,2021-05-18 03:20:00,CASANARE,MANÍ,lune,CENTRAL DE MANI,URBANA,"HOTELES, RESIDENCIAS, Y SIMILARES.",ARMA BLANCA,A PIE,A PIE,27.0,FEMENINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,85139000.0,1988.0,92-874,jgffkn2333@gmail.com,92,874
4587,2021-05-18 10:15:00,VALLE,GUADALAJARA DE BUGA,Lumes,VIA BUGA NO REPORTADO MEDIACANOA,RURAL,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,25.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76111000.0,1990.0,27-345,rjnpih6160@gmail.com,27,345
4971,2021-05-30 13:22:00,VALLE,CALI (CT),Sábado,PUERTAS DEL SOL E14,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,35.0,MASCULINO,UNION LIBRE,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1980.0,55-963,gmfrkj7682@gmail.com,55,963
5372,2021-06-09 10:50:00,NARIÑO,PASTO (CT),Msrtes,RIVIERA,URBANA,RIOS,ARMA BLANCA,A PIE,A PIE,20.0,MASCULINO,NO REPORTADO,NO REPORTA,NO REPORTADO,NO REPORTADO,NO REPORTADO,52001000.0,1995.0,63-185,blhqee9596@unidatos.edu.co,63,185


<span style="color:red">TODO: Make something similar to check and fix the "CORREO" column (PATTERN: 4 digits before the @, only .edu.co and .com domains are allowed)</span>

In [33]:
# Partimos por quitarle los espacios a los correos.
homicides_df["CORREO"].str.strip()

# Luego realizo un split del correo para poder analizar por aparte el dominio y el nombre del correo. 
correo_malformed = homicides_df["CORREO"].str.split(pat='@',n=2, expand=True)
correo_malformed.rename(columns={0:"Direccion",1:"Dominio"}, inplace=True)

# Identifico los correos cuyo dominio es diferente de edu.co o .com . Esto nos da 1.011 registros.

correo_malformed['DominioEDU'] = correo_malformed['Dominio'].str.contains("edu.co", case=False, regex=True)
correo_malformed['DominioCOM'] = correo_malformed['Dominio'].str.contains(".com", case=False, regex=True)
flt = correo_malformed['DominioEDU']==False
correo_malformed = correo_malformed[flt]
flt = correo_malformed['DominioCOM']==False
correo_malformed = correo_malformed[flt]
correo_malformed

,Direccion,Dominio,DominioEDU,DominioCOM
36,correo4714,colombia.gov.co,False,False
47,correo8297,colombia.gov.co,False,False
55,correo7285,colombia.gov.co,False,False
84,correo8342,colombia.gov.co,False,False
104,correo6479,colombia.gov.co,False,False
...,...,...,...,...
12370,correo7745,colombia.gov.co,False,False
12378,correo9786,colombia.gov.co,False,False
12380,correo8859,colombia.gov.co,False,False
12387,correo3558,colombia.gov.co,False,False


In [34]:
# Dado que no hay una forma de solucionarlo y se debe arreglar. Para que solo tenga 4 caracteres antes del @ tomaré los últimos 4 digitos y el dominio será reemplazado por 
# por gmail.com

correo_malformed['CorreoCorrecto']=correo_malformed['Direccion'].str[-4:]+"@gmail.com"
homicides_df["CORREO"].update(correo_malformed['CorreoCorrecto'])
homicides_df

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
0,2021-01-01 06:00:00,ANTIOQUIA,AMAGÁ,Jueves,EL VOLCAN,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,44.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5030000.0,1971.0,42-908,agbnqg2122@unidatos.edu.co
1,2021-01-01 09:00:00,ANTIOQUIA,BARBOSA,Jueves,VDA. MATASANOS,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,NO REPORTADO,5079000.0,1985.0,15-183,rbkeui3584@gmail.com
2,2021-01-01 19:00:00,ANTIOQUIA,EL BAGRE,Jueves,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1982.0,84-786,aorkhf9155@unidatos.edu.co
3,2021-01-01 11:20:00,ANTIOQUIA,JARDÍN,Jueves,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1975.0,31-289,dhtemr6623@unidatos.edu.co
4,2021-01-01 15:00:00,ANTIOQUIA,MEDELLÍN (CT),Juees,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1949.0,66-363,artatj9268@unidatos.edu.co
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12394,2021-12-31 15:42:00,VALLE,EL CERRITO,Jueves,LA ESPERANZA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,53.0,FEMENINO,CASADO,EMPLEADO PARTICULAR,NO REPORTADO,PRIMARIA,COLOMBIA,76248000.0,1962.0,39-658,nggiso7664@gmail.com
12395,2021-12-31 19:04:00,VALLE,LA CUMBRE,Jueves,CGTO BITACO E27,RURAL,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,58.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,76377000.0,1957.0,39-442,4352@gmail.com
12396,2021-12-31 22:35:00,VALLE,PALMIRA,Jueves,LORETO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,18.0,FEMENINO,SOLTERO,ESTUDIANTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76520000.0,1997.0,25-368,fktatq9245@unidatos.edu.co
12397,2021-12-31 14:30:00,VALLE,ZARZAL,Jueves,URB. EL ZARZALITO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,27.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76895000.0,1988.0,62-735,gafkup6393@gmail.com


In [35]:
# Showing different values for "DIA" column
homicides_df["DIA"].unique()

array(['Jueves', 'Juees', 'Jueces', 'juves', 'Juevrs', 'Viernes',
       'Viermes', 'iernes', 'virnes', 'Vierens', 'Sábado', 'Sabadi',
       'Sabado', 'sábad', 'Sávado', 'Ssbado', 'Domingo', 'Domungo',
       'Doningo', 'domungo', 'Lunes', 'lune', 'Luns', 'Lumes', 'kunes',
       'Lnues', 'Martes', 'Mates', 'Marte', 'mates', 'Miércoles',
       'Miwrcoles', 'Mircoles', 'Voernes', 'domnigo', 'Maryes',
       'Miercoles', 'miércles', 'Dominog', 'Msrtes', 'Mirrcoles'],
      dtype=object)

In [36]:
# Calculating the distance between two words using the Levenshtein method
pylev.levenshtein('sábado', 'sabaod')

3

In [37]:
pylev.levenshtein('sábado', 'viernes')

7

<span style="color:red">TODO: Create a function to fix the digitation errors for "DIA" column.</span>

In [38]:
# Primero, voy a experimentar un poco con la función verificando valores como Viernes con iernes
pylev.levenshtein('viernes', 'iernes')
pylev.levenshtein('Jueves', 'Juees')
pylev.levenshtein('Lunes', 'Juees')

2

In [39]:
# Convierto el día a mayusculas para eliminar posibles diferencias ocasionadas por las letras capitales

homicides_df["DIA"]=homicides_df["DIA"].str.upper()
homicides_df["DIA"].unique()

array(['JUEVES', 'JUEES', 'JUECES', 'JUVES', 'JUEVRS', 'VIERNES',
       'VIERMES', 'IERNES', 'VIRNES', 'VIERENS', 'SÁBADO', 'SABADI',
       'SABADO', 'SÁBAD', 'SÁVADO', 'SSBADO', 'DOMINGO', 'DOMUNGO',
       'DONINGO', 'LUNES', 'LUNE', 'LUNS', 'LUMES', 'KUNES', 'LNUES',
       'MARTES', 'MATES', 'MARTE', 'MIÉRCOLES', 'MIWRCOLES', 'MIRCOLES',
       'VOERNES', 'DOMNIGO', 'MARYES', 'MIERCOLES', 'MIÉRCLES', 'DOMINOG',
       'MSRTES', 'MIRRCOLES'], dtype=object)

In [40]:
# Se define una función utilizando el metodo Levenshtein para corregir las diferencias solo en grado 1. Se hace paulatino para que palabras como Juees que no dista
# mucho del Lunes, pueda ser corregida por Jueves y no por el primero que encuentre con grado mayor.

def corregirErrorDiaGrado1(x):
    distancia = 1
    if pylev.levenshtein('LUNES', x) <= distancia: 
        return 'LUNES'
    elif pylev.levenshtein('MARTES', x) <= distancia:
        return 'MARTES'
    elif pylev.levenshtein('MIÉRCOLES', x) <= distancia:
        return 'MIERCOLES'
    elif pylev.levenshtein('JUEVES', x) <= distancia:
        return 'JUEVES'
    elif pylev.levenshtein('VIERNES', x) <= distancia:
        return 'VIERNES'
    elif pylev.levenshtein('SÁBADO', x) <= distancia:
        return 'SABADO'
    elif pylev.levenshtein('DOMINGO', x) <= distancia:
        return 'DOMINGO'
    else:
        return x

homicides_df["DIA"]=homicides_df["DIA"].apply(corregirErrorDiaGrado1)

In [41]:
homicides_df["DIA"].unique()

array(['JUEVES', 'VIERNES', 'VIERENS', 'SABADO', 'SABADI', 'DOMINGO',
       'LUNES', 'LNUES', 'MARTES', 'MIERCOLES', 'DOMNIGO', 'DOMINOG'],
      dtype=object)

In [42]:
# Se define una función utilizando el metodo Levenshtein para corregir las diferencias en grado 2.

def corregirErrorDiaGrado2(x):
    distancia = 2
    if pylev.levenshtein('LUNES', x) <= distancia: 
        return 'LUNES'
    elif pylev.levenshtein('MARTES', x) <= distancia:
        return 'MARTES'
    elif pylev.levenshtein('MIÉRCOLES', x) <= distancia:
        return 'MIERCOLES'
    elif pylev.levenshtein('JUEVES', x) <= distancia:
        return 'JUEVES'
    elif pylev.levenshtein('VIERNES', x) <= distancia:
        return 'VIERNES'
    elif pylev.levenshtein('SÁBADO', x) <= distancia:
        return 'SABADO'
    elif pylev.levenshtein('DOMINGO', x) <= distancia:
        return 'DOMINGO'
    else:
        return x

homicides_df["DIA"]=homicides_df["DIA"].apply(corregirErrorDiaGrado2)
homicides_df["DIA"].unique()

array(['JUEVES', 'VIERNES', 'SABADO', 'DOMINGO', 'LUNES', 'MARTES',
       'MIERCOLES'], dtype=object)

In [43]:
homicides_df[["AÑO DE NACIMIENTO", "EDAD"]].head(10)

,AÑO DE NACIMIENTO,EDAD
0,1971.0,44.0
1,1985.0,30.0
2,1982.0,33.0
3,1975.0,40.0
4,1949.0,66.0
5,1973.0,42.0
6,1990.0,25.0
7,1978.0,37.0
8,1990.0,25.0
9,1985.0,30.0


<span style="color:red">TODO: Fix the "AÑO DE NACIMIENTO" column using the column "EDAD".</span>

In [44]:
homicides_df["AÑO DE NACIMIENTO"]=2022-homicides_df["EDAD"]

In [45]:
homicides_df

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
0,2021-01-01 06:00:00,ANTIOQUIA,AMAGÁ,JUEVES,EL VOLCAN,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,44.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5030000.0,1978.0,42-908,agbnqg2122@unidatos.edu.co
1,2021-01-01 09:00:00,ANTIOQUIA,BARBOSA,JUEVES,VDA. MATASANOS,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,NO REPORTADO,5079000.0,1992.0,15-183,rbkeui3584@gmail.com
2,2021-01-01 19:00:00,ANTIOQUIA,EL BAGRE,JUEVES,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1989.0,84-786,aorkhf9155@unidatos.edu.co
3,2021-01-01 11:20:00,ANTIOQUIA,JARDÍN,JUEVES,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1982.0,31-289,dhtemr6623@unidatos.edu.co
4,2021-01-01 15:00:00,ANTIOQUIA,MEDELLÍN (CT),JUEVES,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1956.0,66-363,artatj9268@unidatos.edu.co
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12394,2021-12-31 15:42:00,VALLE,EL CERRITO,JUEVES,LA ESPERANZA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,53.0,FEMENINO,CASADO,EMPLEADO PARTICULAR,NO REPORTADO,PRIMARIA,COLOMBIA,76248000.0,1969.0,39-658,nggiso7664@gmail.com
12395,2021-12-31 19:04:00,VALLE,LA CUMBRE,JUEVES,CGTO BITACO E27,RURAL,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,58.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,76377000.0,1964.0,39-442,4352@gmail.com
12396,2021-12-31 22:35:00,VALLE,PALMIRA,JUEVES,LORETO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,18.0,FEMENINO,SOLTERO,ESTUDIANTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76520000.0,2004.0,25-368,fktatq9245@unidatos.edu.co
12397,2021-12-31 14:30:00,VALLE,ZARZAL,JUEVES,URB. EL ZARZALITO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,27.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76895000.0,1995.0,62-735,gafkup6393@gmail.com
